## Newton para Sistemas não Lineares

O método de Newton para sistemas não lineares busca encontrar a solução de um sistema f(x) = 0 utilizando aproximações sucessivas e a inversa do Jacobiano.


In [1]:
import numpy as np
import sympy as sp
from tabulate import tabulate

In [7]:
def newton_sistemas(funcs, vars, x0, tol, max_iter=100):
    """
    Resolve sistemas de equações não lineares pelo método de Newton.

    Parâmetros:
    funcs -- Lista de funções do sistema (expressões simbólicas)
    vars -- Lista de variáveis
    x0 -- Aproximação inicial (array)
    tol -- Erro de tolerância
    max_iter -- Número máximo de iterações

    Retorna:
    Solução aproximada e tabela com as iterações
    """
    J = sp.Matrix([[sp.diff(f, var) for var in vars] for f in funcs])  # Jacobiano
    
    f_lambdify = sp.lambdify(vars, funcs, 'numpy')
    J_lambdify = sp.lambdify(vars, J, 'numpy')

    x = np.array(x0, dtype=float)
    tabela = []

    for iter in range(max_iter):
        F_val = np.array(f_lambdify(*x), dtype=float).flatten()
        J_val = np.array(J_lambdify(*x), dtype=float)

        if np.linalg.cond(J_val) > 1e12:  # Verifica se o Jacobiano está próximo da singularidade
            print("Jacobiano mal condicionado! Pode não haver convergência.")
            return None, tabela

        try:
            delta_x = np.linalg.solve(J_val, -F_val)  # Resolve o sistema J * delta_x = -F
        except np.linalg.LinAlgError:
            print("Erro: Jacobiano singular ou indefinido.")
            return None, tabela

        x_new = x + delta_x

        erro_absoluto = np.linalg.norm(delta_x)
        erro_relativo = erro_absoluto / (np.linalg.norm(x_new) if np.linalg.norm(x_new) != 0 else 1)

        tabela.append([iter, *x, erro_absoluto, erro_relativo])

        if erro_absoluto <= tol:
            return np.round(x_new, 4), tabela
        
        x = x_new

In [8]:
# Entrada do usuário
n = int(input("Digite o número de equações/variáveis: "))
vars = sp.symbols(f'x1:{n+1}')
funcs = [sp.sympify("-10*x1+(25*cos(x1))+10*x2-10*x1"), sp.sympify("-10*x2+10*x1+(15*cos(x2))+10*x3-10*x2"), sp.sympify("10*x3-10*x2-(5*cos(x3))")]

# print("Digite as funções do sistema:")
# for i in range(n):
    # funcs.append(sp.sympify(input(f"f{i+1}(x1, ..., xn) = ")))

x0 = [float(input(f"Digite x{i+1} inicial: ")) for i in range(n)]
tol = float(input("Digite o erro de tolerância: "))

resultado = newton_sistemas(funcs, vars, x0, tol)

if resultado is None:
    print("O método não convergiu ou houve um erro numérico.")
else:
    solucao, tabela = resultado
    
    # Exibindo tabela de resultados
    headers = ["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro Absoluto", "Erro Relativo"]
    print(tabulate(tabela, headers=headers, floatfmt=".4f"))
    
    print(f"Solução aproximada: {solucao}")

TypeError: cannot unpack non-iterable NoneType object